Extract monthly `.osm.pbf` files 

install `osmium-tool` 

```shell
conda install osmium-tool
```

```shell
activate py310geo

cd "C:\Users\Wei Zhou\OneDrive - National University of Singapore\Pesearch Project\Traffic dataset\Open street map\OSM Singapore"

for %y in (2020, 2021, 2022, 2023) do (
	for %m in (01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12) do (
	    ECHO "Extracting temporal OSM data for %y-%m"    
	    osmium time-filter ^
	        -o "extract\malaysia-singapore-brunei-%y-%m-28.osm.pbf" ^
	        "osm-raw-data-sg\malaysia-singapore-brunei-internal-2024-10-28.osh.pbf" ^
	        %y-%m-28T00:00:00Z
	    
	    ECHO "Extracting OSM data within the boundary"
	    osmium extract ^
	        -p "osm-raw-data-sg\MP2019-boundary-with-sea.poly" ^
	        -o "extract\singapore_%y-%m-28.osm.pbf" ^
	        -s complete_ways ^
	        "extract\malaysia-singapore-brunei-%y-%m-28.osm.pbf"
	)
)
```

In [1]:
import pathlib

import pyrosm
import tqdm

In [2]:
osm_folder = r'C:\Users\Wei Zhou\OneDrive - National University of Singapore\Pesearch Project\Traffic dataset\Open street map\OSM Singapore\extract'
osm_path_li = pathlib.Path(osm_folder).glob('*.osm.pbf')
osm_path_li = [p.absolute() for p in osm_path_li]

In [3]:
data_all = []

for osm_path in tqdm.tqdm(osm_path_li, total=len(osm_path_li), desc='Extracting POIs:'):
    # initialize the OSM object
    osm_map = pyrosm.OSM(str(osm_path))
    # get all POIs
    # default poi types: ['amenity', 'shop', 'tourism']
    tags_filter = {'amenity': True, 'shop': True, 'tourism': True, 'leisure': True}
    pois = osm_map.get_pois(tags_filter)
    # selected necessary columns
    pois = pois[['id', 'osm_type', 'addr:postcode', 'addr:street', 'geometry'] + list(tags_filter.keys())]
    # save the data
    _dt = osm_path.stem.replace('singapore_', '').replace('.osm', '')
    pois.to_file(f'data/osm_poi_{_dt}.geojson', driver='GeoJSON')
# data_all = pd.concat(data_all, ignore_index=True, axis=0)

Extracting POIs:: 100%|██████████| 48/48 [27:53<00:00, 34.87s/it]
